In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import random

# Set up data paths
data_dir = Path('../data')
usda_dir = data_dir / 'FoodData_Central_csv_2025-04-24'

print("Loading datasets...")

Loading datasets...


In [2]:
# Load the main datasets
metadata_df = pd.read_csv(data_dir / 'Metadata_500food.csv')
intensity_df = pd.read_csv(data_dir / 'featuretable_reformated - Kushal.csv')

# Load USDA datasets including sr_legacy_food
food_df = pd.read_csv(usda_dir / 'food.csv')
sr_legacy_food_df = pd.read_csv(usda_dir / 'sr_legacy_food.csv')
food_nutrient_df = pd.read_csv(usda_dir / 'food_nutrient.csv')
nutrient_df = pd.read_csv(usda_dir / 'nutrient.csv')

print(f"Metadata shape: {metadata_df.shape}")
print(f"Intensity matrix shape: {intensity_df.shape}")
print(f"USDA food shape: {food_df.shape}")
print(f"SR Legacy food shape: {sr_legacy_food_df.shape}")
print(f"USDA food_nutrient shape: {food_nutrient_df.shape}")
print(f"USDA nutrient shape: {nutrient_df.shape}")

/var/folders/vl/nn1r9fg115dd94js8h25w1yc0000gn/T/ipykernel_12953/2645093397.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient_df = pd.read_csv(usda_dir / 'food_nutrient.csv')


Metadata shape: (500, 159)
Intensity matrix shape: (54546, 526)
USDA food shape: (2064912, 5)
SR Legacy food shape: (7793, 2)
USDA food_nutrient shape: (26805037, 13)
USDA nutrient shape: (477, 5)


In [3]:
# Analyze metadata structure
print("=== METADATA ANALYSIS ===")
print(f"Unique sample_type_group5 values: {metadata_df['sample_type_group5'].nunique()}")
print(f"Unique filenames: {metadata_df['filename'].nunique()}")

# Show sample_type_group5 distribution
print("\nSample_type_group5 distribution (top 10):")
group5_counts = metadata_df['sample_type_group5'].value_counts().head(10)
for food, count in group5_counts.items():
    print(f"  {food}: {count} samples")

# Check ndb_number availability and data type
print(f"\nndb_number info:")
print(f"Non-null count: {metadata_df['ndb_number'].notna().sum()}")
print(f"Unique ndb_numbers: {metadata_df['ndb_number'].nunique()}")
print(f"Data type: {metadata_df['ndb_number'].dtype}")
print(f"Sample values: {metadata_df['ndb_number'].dropna().head().tolist()}")

# Show some example mappings
print("\nSample filename to food mappings:")
sample_data = metadata_df[['filename', 'sample_type_group5', 'ndb_number', 'ndb_description']].head(10)
for _, row in sample_data.iterrows():
    print(f"{row['filename']} -> {row['sample_type_group5']} (ndb: {row['ndb_number']})")

=== METADATA ANALYSIS ===
Unique sample_type_group5 values: 141
Unique filenames: 500

Sample_type_group5 distribution (top 10):
  tea: 51 samples
  beef: 34 samples
  mouse: 14 samples
  trout: 13 samples
  pepper: 13 samples
  orange: 13 samples
  lettuce: 11 samples
  coffee: 11 samples
  soda: 10 samples
  tomato: 10 samples

ndb_number info:
Non-null count: 500
Unique ndb_numbers: 159
Data type: object
Sample values: ['14003', '45182628', '11215', '45096876', '11098']

Sample filename to food mappings:
P3_E8_G72464.mzML -> grain_fermented (ndb: 14003)
P5_G5_G72471.mzML -> pea (ndb: 45182628)
P3_C12_72475.mzML -> garlic (ndb: 11215)
P3_B9_G72492.mzML -> raspberry (ndb: 45096876)
P3_B8_G72493.mzML -> brussel sprout (ndb: 11098)
P3_B7_72497.mzML -> melon (ndb: 9181)
P3_B6_G72498.mzML -> pepper (ndb: 11979)
P3_B5_72500.mzML -> apple (ndb: 9020)
P5_G4_G72502.mzML -> chia (ndb: 45120594)
P5_G2_G72503.mzML -> oat (ndb: 45057949)


In [4]:
# Analyze the SR Legacy food mapping
print("=== SR LEGACY FOOD MAPPING ANALYSIS ===")
print(f"SR Legacy food unique fdc_ids: {sr_legacy_food_df['fdc_id'].nunique()}")
print(f"SR Legacy food unique NDB numbers: {sr_legacy_food_df['NDB_number'].nunique()}")

# Check the mapping structure
print("\nSR Legacy food columns:")
print(sr_legacy_food_df.columns.tolist())

# Check NDB_number data type
print(f"\nNDB_number data type: {sr_legacy_food_df['NDB_number'].dtype}")
print(f"Sample NDB_number values: {sr_legacy_food_df['NDB_number'].dropna().head().tolist()}")

# Show some example mappings
print("\nSample SR Legacy food mappings:")
sample_sr = sr_legacy_food_df[['fdc_id', 'NDB_number']].head(10)
for _, row in sample_sr.iterrows():
    print(f"fdc_id {row['fdc_id']} -> NDB_number {row['NDB_number']}")

# Check for our metadata ndb_numbers in SR Legacy (with type conversion)
metadata_ndb_numbers = metadata_df['ndb_number'].dropna().astype(str).unique()
print(f"\nOur metadata has {len(metadata_ndb_numbers)} unique NDB numbers (as strings)")

# Find matches in SR Legacy
sr_ndb_numbers = sr_legacy_food_df['NDB_number'].dropna().astype(str).unique()
matches = set(metadata_ndb_numbers).intersection(set(sr_ndb_numbers))
print(f"Found {len(matches)} NDB numbers that match between metadata and SR Legacy")

if matches:
    print("Sample matching NDB numbers:")
    for ndb in list(matches)[:5]:
        print(f"  NDB {ndb}")

=== SR LEGACY FOOD MAPPING ANALYSIS ===
SR Legacy food unique fdc_ids: 7793
SR Legacy food unique NDB numbers: 7793

SR Legacy food columns:
['fdc_id', 'NDB_number']

NDB_number data type: int64
Sample NDB_number values: [18634, 18635, 18637, 18639, 18932]

Sample SR Legacy food mappings:
fdc_id 167512 -> NDB_number 18634
fdc_id 167513 -> NDB_number 18635
fdc_id 167514 -> NDB_number 18637
fdc_id 167515 -> NDB_number 18639
fdc_id 167516 -> NDB_number 18932
fdc_id 167517 -> NDB_number 18933
fdc_id 167518 -> NDB_number 18934
fdc_id 167519 -> NDB_number 18935
fdc_id 167520 -> NDB_number 18942
fdc_id 167521 -> NDB_number 18943

Our metadata has 159 unique NDB numbers (as strings)
Found 103 NDB numbers that match between metadata and SR Legacy
Sample matching NDB numbers:
  NDB 11241
  NDB 15085
  NDB 2038
  NDB 11098
  NDB 9156


In [5]:
# Analyze intensity matrix
print("=== INTENSITY MATRIX ANALYSIS ===")
print(f"Features (rows): {intensity_df.shape[0]}")
print(f"Samples (columns): {intensity_df.shape[1]}")

# Check if there's a Feature column
if 'Feature' in intensity_df.columns:
    print(f"\nFeature column found with {intensity_df['Feature'].nunique()} unique features")
    print("Sample features:", intensity_df['Feature'].head().tolist())
else:
    print("\nNo 'Feature' column found. Checking first column...")
    print("First column name:", intensity_df.columns[0])
    print("Sample values:", intensity_df.iloc[:5, 0].tolist())

# Show column structure
print(f"\nColumn names (first 10):")
print(intensity_df.columns[:10].tolist())

# Check for peak area columns (both cases)
peak_area_cols = [col for col in intensity_df.columns if 'peak area' in col.lower()]
print(f"\nFound {len(peak_area_cols)} peak area columns")
if peak_area_cols:
    print("Sample peak area columns:", peak_area_cols[:5])

=== INTENSITY MATRIX ANALYSIS ===
Features (rows): 54546
Samples (columns): 526

Feature column found with 54546 unique features
Sample features: [271, 4396, 18754, 15057, 125441]

Column names (first 10):
['Feature', 'P4_C10_G75213.mzML Peak area', 'P4_F3_G75184.mzML Peak area', 'P3_F1_G74067.mzML Peak area', 'P4_D9_G75202.mzML Peak area', 'P2_F5_G83240.mzML Peak area', 'P1_B1_G75794.mzML Peak area', 'P2_E12_G83237.mzML Peak area', 'P2_D8_G87461.mzML Peak area', 'G73730.mzML Peak area']

Found 524 peak area columns
Sample peak area columns: ['P4_C10_G75213.mzML Peak area', 'P4_F3_G75184.mzML Peak area', 'P3_F1_G74067.mzML Peak area', 'P4_D9_G75202.mzML Peak area', 'P2_F5_G83240.mzML Peak area']


In [6]:
# Pick a couple of random foods for testing
print("=== SELECTING RANDOM FOODS FOR TESTING ===")

# Get unique foods from sample_type_group5
unique_foods = metadata_df['sample_type_group5'].dropna().unique()
print(f"Total unique foods: {len(unique_foods)}")

# Pick 3 random foods
random_foods = random.sample(list(unique_foods), 3)
print(f"\nSelected random foods: {random_foods}")

# Get all samples for these foods
test_foods_data = metadata_df[metadata_df['sample_type_group5'].isin(random_foods)]
print(f"\nFound {len(test_foods_data)} samples for these foods:")
for food in random_foods:
    food_samples = test_foods_data[test_foods_data['sample_type_group5'] == food]
    print(f"  {food}: {len(food_samples)} samples")

=== SELECTING RANDOM FOODS FOR TESTING ===
Total unique foods: 141

Selected random foods: ['sweet potato', 'langostino', 'kohlrabi']

Found 9 samples for these foods:
  sweet potato: 6 samples
  langostino: 2 samples
  kohlrabi: 1 samples


In [7]:
# Test intensity linking for one food
print("=== TESTING INTENSITY LINKING ===")

test_food = random_foods[0]
print(f"Testing with food: {test_food}")

# Get all samples for this food
food_samples = metadata_df[metadata_df['sample_type_group5'] == test_food]
print(f"Found {len(food_samples)} samples")

# Get the first sample's filename
sample_filename = food_samples.iloc[0]['filename']
print(f"Using sample filename: {sample_filename}")

# Look for the intensity column (try both cases)
intensity_col_lower = f"{sample_filename} peak area"
intensity_col_upper = f"{sample_filename} Peak area"

print(f"Looking for intensity columns:")
print(f"  Lower case: {intensity_col_lower}")
print(f"  Upper case: {intensity_col_upper}")

# Check which one exists
if intensity_col_lower in intensity_df.columns:
    intensity_col = intensity_col_lower
    print("✓ Found lower case intensity column!")
elif intensity_col_upper in intensity_df.columns:
    intensity_col = intensity_col_upper
    print("✓ Found upper case intensity column!")
else:
    print("✗ Neither intensity column found")
    # Show available columns that might match
    matching_cols = [col for col in intensity_df.columns if sample_filename in col]
    print(f"Available columns containing '{sample_filename}': {matching_cols}")
    intensity_col = None

if intensity_col:
    # Get all feature intensities for this sample
    feature_intensities = intensity_df[['Feature', intensity_col]].copy()
    feature_intensities = feature_intensities.rename(columns={intensity_col: 'intensity'})
    
    # Remove rows where intensity is 0 or NaN
    feature_intensities = feature_intensities[
        (feature_intensities['intensity'] > 0) & 
        (feature_intensities['intensity'].notna())
    ]
    
    print(f"Found {len(feature_intensities)} features with non-zero intensity")
    print(f"Intensity range: {feature_intensities['intensity'].min():.2f} to {feature_intensities['intensity'].max():.2f}")
    
    # Show top features by intensity
    print("\nTop 10 features by intensity:")
    top_features = feature_intensities.nlargest(10, 'intensity')
    for _, row in top_features.iterrows():
        print(f"  Feature {row['Feature']}: {row['intensity']:.2f}")

=== TESTING INTENSITY LINKING ===
Testing with food: sweet potato
Found 6 samples
Using sample filename: P2_E1_G87553.mzML
Looking for intensity columns:
  Lower case: P2_E1_G87553.mzML peak area
  Upper case: P2_E1_G87553.mzML Peak area
✓ Found upper case intensity column!
Found 10516 features with non-zero intensity
Intensity range: 12.35 to 91609544.00

Top 10 features by intensity:
  Feature 1754.0: 91609544.00
  Feature 1065.0: 75991500.00
  Feature 3407.0: 23949620.00
  Feature 1638.0: 15438614.00
  Feature 1409.0: 12766388.00
  Feature 2420.0: 9508895.00
  Feature 942.0: 8696197.00
  Feature 1474.0: 8645326.00
  Feature 7428.0: 8447453.00
  Feature 247010.0: 7921888.00


In [8]:
# Test USDA nutritional data linking
print("=== TESTING USDA NUTRITIONAL DATA LINKING ===")

# Get a sample with ndb_number
sample_with_ndb = metadata_df[metadata_df['ndb_number'].notna()].iloc[45]
test_food = sample_with_ndb['sample_type_group5']
ndb_number = sample_with_ndb['ndb_number']

print(f"Testing with food: {test_food}")
print(f"NDB Number: {ndb_number} (type: {type(ndb_number)})")

# Step 1: Find fdc_id in SR Legacy food (convert to string for comparison)
sr_legacy_match = sr_legacy_food_df[sr_legacy_food_df['NDB_number'].astype(str) == str(ndb_number)]

if not sr_legacy_match.empty:
    fdc_id = sr_legacy_match.iloc[0]['fdc_id']
    print(f"✓ Found SR Legacy match: fdc_id {fdc_id}")
    
    # Step 2: Get nutrient data for this fdc_id
    food_nutrients = food_nutrient_df[food_nutrient_df['fdc_id'] == fdc_id]
    
    if not food_nutrients.empty:
        print(f"✓ Found {len(food_nutrients)} nutrient entries")
        
        # Step 3: Get nutrient names
        nutrient_ids = food_nutrients['nutrient_id'].unique()
        nutrient_names = nutrient_df[nutrient_df['id'].isin(nutrient_ids)]
        
        # Show top nutrients
        merged_nutrients = food_nutrients.merge(
            nutrient_names[['id', 'name', 'unit_name']], 
            left_on='nutrient_id', 
            right_on='id'
        )
        
        print(f"\nNutrient data for {test_food}:")
        for _, nutrient in merged_nutrients.head(10).iterrows():
            print(f"  {nutrient['name']}: {nutrient['amount']} {nutrient['unit_name']}")
    else:
        print("✗ No nutrient data found")
else:
    print(f"✗ No SR Legacy match for NDB {ndb_number}")
    # Show some SR Legacy NDB numbers for debugging
    print("Sample SR Legacy NDB numbers:", sr_legacy_food_df['NDB_number'].dropna().head().tolist())

=== TESTING USDA NUTRITIONAL DATA LINKING ===
Testing with food: potato
NDB Number: 11365 (type: <class 'str'>)
✓ Found SR Legacy match: fdc_id 170438
✓ Found 98 nutrient entries

Nutrient data for potato:
  SFA 4:0: 0.0 G
  PUFA 18:3: 0.01 G
  Carotene, alpha: 0.0 UG
  Fluoride, F: 49.4 UG
  PUFA 18:4: 0.0 G
  Vitamin E (alpha-tocopherol): 0.01 MG
  Potassium, K: 379.0 MG
  SFA 12:0: 0.003 G
  Carbohydrate, by difference: 20.13 G
  PUFA 22:6 n-3 (DHA): 0.0 G


In [9]:
# Create functions for complete linking
def get_food_intensities(food_name, intensity_df, metadata_df):
    """
    Get all feature intensities for all samples of a given food.
    
    Args:
        food_name: Name of the food from sample_type_group5
        intensity_df: Feature intensity matrix
        metadata_df: Food metadata
    
    Returns:
        DataFrame with Feature, filename, intensity columns
    """
    # Get all samples for this food
    food_samples = metadata_df[metadata_df['sample_type_group5'] == food_name]
    
    if food_samples.empty:
        print(f"No samples found for food: {food_name}")
        return pd.DataFrame()
    
    all_intensities = []
    
    for _, sample in food_samples.iterrows():
        filename = sample['filename']
        
        # Try both cases for peak area column
        intensity_col_lower = f"{filename} peak area"
        intensity_col_upper = f"{filename} Peak area"
        
        intensity_col = None
        if intensity_col_lower in intensity_df.columns:
            intensity_col = intensity_col_lower
        elif intensity_col_upper in intensity_df.columns:
            intensity_col = intensity_col_upper
        
        if intensity_col:
            # Get intensities for this sample
            sample_intensities = intensity_df[['Feature', intensity_col]].copy()
            sample_intensities = sample_intensities.rename(columns={intensity_col: 'intensity'})
            sample_intensities['filename'] = filename
            
            # Filter non-zero intensities
            sample_intensities = sample_intensities[
                (sample_intensities['intensity'] > 0) & 
                (sample_intensities['intensity'].notna())
            ]
            
            all_intensities.append(sample_intensities)
    
    if all_intensities:
        result = pd.concat(all_intensities, ignore_index=True)
        print(f"Found {len(result)} feature-intensity pairs for {food_name}")
        return result
    else:
        print(f"No intensity data found for {food_name}")
        return pd.DataFrame()

def get_food_nutrients_with_sr_legacy(food_name, metadata_df, sr_legacy_food_df, food_nutrient_df, nutrient_df):
    """
    Get nutritional data for a food using NDB number -> SR Legacy -> fdc_id linking.
    
    Args:
        food_name: Name of the food
        metadata_df: Food metadata with NDB numbers
        sr_legacy_food_df: SR Legacy food mapping
        food_nutrient_df: USDA food_nutrient.csv
        nutrient_df: USDA nutrient.csv
    
    Returns:
        DataFrame with nutrient information
    """
    # Get NDB number for this food
    food_metadata = metadata_df[metadata_df['sample_type_group5'] == food_name]
    ndb_numbers = food_metadata['ndb_number'].dropna().unique()
    
    if len(ndb_numbers) == 0:
        print(f"No NDB numbers found for {food_name}")
        return pd.DataFrame()
    
    # Use the first NDB number
    ndb_number = ndb_numbers[0]
    print(f"Looking for NDB number {ndb_number} for {food_name}")
    
    # Step 1: Find fdc_id in SR Legacy food (convert to string for comparison)
    sr_legacy_match = sr_legacy_food_df[sr_legacy_food_df['NDB_number'].astype(str) == str(ndb_number)]
    
    if sr_legacy_match.empty:
        print(f"No SR Legacy match for NDB {ndb_number}")
        return pd.DataFrame()
    
    fdc_id = sr_legacy_match.iloc[0]['fdc_id']
    print(f"✓ Found SR Legacy match: fdc_id {fdc_id}")
    
    # Step 2: Get nutrient data for this fdc_id
    food_nutrients = food_nutrient_df[food_nutrient_df['fdc_id'] == fdc_id]
    
    if food_nutrients.empty:
        print(f"No nutrient data found for fdc_id {fdc_id}")
        return pd.DataFrame()
    
    print(f"✓ Found {len(food_nutrients)} nutrient entries")
    
    # Step 3: Get nutrient names
    nutrient_ids = food_nutrients['nutrient_id'].unique()
    nutrient_names = nutrient_df[nutrient_df['id'].isin(nutrient_ids)]
    
    # Merge nutrient data with names
    result = food_nutrients.merge(
        nutrient_names[['id', 'name', 'unit_name']], 
        left_on='nutrient_id', 
        right_on='id'
    )
    
    print(f"✓ Found {len(result)} nutrients with names")
    return result[['name', 'amount', 'unit_name']]

# Test the functions
print("=== TESTING COMPLETE LINKING FUNCTIONS ===")

# Get foods with NDB numbers
foods_with_ndb = metadata_df[metadata_df['ndb_number'].notna()]['sample_type_group5'].unique()
if len(foods_with_ndb) > 0:
    test_foods = random.sample(list(foods_with_ndb), min(3, len(foods_with_ndb)))
else:
    test_foods = random_foods

for food in test_foods:
    print(f"\n--- Testing {food} ---")
    
    # Get intensities
    intensities = get_food_intensities(food, intensity_df, metadata_df)
    if not intensities.empty:
        print(f"Intensity summary: {len(intensities)} features, {intensities['filename'].nunique()} samples")
    
    # Get nutrients
    nutrients = get_food_nutrients_with_sr_legacy(
        food, metadata_df, sr_legacy_food_df, food_nutrient_df, nutrient_df
    )
    if not nutrients.empty:
        print("Top nutrients:")
        for _, nutrient in nutrients.head(5).iterrows():
            print(f"  {nutrient['name']}: {nutrient['amount']} {nutrient['unit_name']}")

=== TESTING COMPLETE LINKING FUNCTIONS ===

--- Testing avocado ---
Found 23580 feature-intensity pairs for avocado
Intensity summary: 23580 features, 2 samples
Looking for NDB number 9037 for avocado
✓ Found SR Legacy match: fdc_id 171705
✓ Found 122 nutrient entries
✓ Found 122 nutrients with names
Top nutrients:
  Fructose: 0.12 G
  Carbohydrate, by difference: 8.53 G
  Fatty acids, total monounsaturated: 9.799 G
  Tocopherol, beta: 0.05 MG
  Copper, Cu: 0.19 MG

--- Testing persimmon ---
Found 19560 feature-intensity pairs for persimmon
Intensity summary: 19560 features, 2 samples
Looking for NDB number 45299869 for persimmon
No SR Legacy match for NDB 45299869

--- Testing banana ---
Found 17478 feature-intensity pairs for banana
Intensity summary: 17478 features, 2 samples
Looking for NDB number 9040 for banana
✓ Found SR Legacy match: fdc_id 173944
✓ Found 108 nutrient entries
✓ Found 108 nutrients with names
Top nutrients:
  Manganese, Mn: 0.27 MG
  Tocopherol, gamma: 0.02 MG
 

In [10]:
print("=== SUMMARY AND NEXT STEPS ===")
print("\nCorrected data linking process:")
print("1. Food name (sample_type_group5) -> multiple filenames (samples)")
print("2. Filename -> <filename> peak area / Peak area column in intensity matrix")
print("3. Feature ID + filename -> intensity value")
print("4. NDB number (int) -> SR Legacy food (NDB_number as str) -> fdc_id -> nutrient data")

print("\nKey corrections:")
print("- Handle both 'peak area' and 'Peak area' column names")
print("- Convert NDB number types: metadata (int) -> SR Legacy (str)")
print("- Use SR Legacy mapping: NDB_number -> fdc_id")

print("\nThis creates the graph structure:")
print("- Molecule nodes: Feature IDs with intensity values")
print("- Food nodes: sample_type_group5 with nutritional profiles")
print("- Edges: (Feature, 'found_in', Food) with intensity weights")

print("\nNext steps:")
print("- Generate Spec2Vec embeddings for Feature IDs")
print("- Create nutritional feature vectors for foods")
print("- Build PyTorch Geometric HeteroData object")
print("- Implement HAN or GIN architecture")

=== SUMMARY AND NEXT STEPS ===

Corrected data linking process:
1. Food name (sample_type_group5) -> multiple filenames (samples)
2. Filename -> <filename> peak area / Peak area column in intensity matrix
3. Feature ID + filename -> intensity value
4. NDB number (int) -> SR Legacy food (NDB_number as str) -> fdc_id -> nutrient data

Key corrections:
- Handle both 'peak area' and 'Peak area' column names
- Convert NDB number types: metadata (int) -> SR Legacy (str)
- Use SR Legacy mapping: NDB_number -> fdc_id

This creates the graph structure:
- Molecule nodes: Feature IDs with intensity values
- Food nodes: sample_type_group5 with nutritional profiles
- Edges: (Feature, 'found_in', Food) with intensity weights

Next steps:
- Generate Spec2Vec embeddings for Feature IDs
- Create nutritional feature vectors for foods
- Build PyTorch Geometric HeteroData object
- Implement HAN or GIN architecture
